### --- Day 17: Clumsy Crucible ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/17

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2023, 17);
WriteLines(inputLines);

Loading puzzle file: Day17.txt
Total lines: 141
Max line length: 141

131445551112233253211343462645362264333536364313674124314476621242354636547511555665246535766355535443144555253111262426511545413534355213143
233424145115554544563536623551634234245245226647466342141335242732317413264173727553773417346271273452636214526231261626636515443114231334134
142542344522223413612323412633411265511751721337441417257213276727474627134235456623341427634774763161362433662654321313511364221333134444545
215325514111432316211413445125445152444115313243425213457743777551554161363342731541663254115427451521737561133533312121462215531232432435315
451531341324135344542462126654325246235214355273527352517361157226542614511154755512673466615546552747525176152644433635456345422344511151123


In [4]:
static byte ParseByte(char ch) => ch switch {
    '0' => 0,
    '1' => 1,
    '2' => 2,
    '3' => 3,
    '4' => 4,
    '5' => 5,
    '6' => 6,
    '7' => 7,
    '8' => 8,
    '9' => 9,
    'x' => 50, // for testing
    _ => throw new Exception("Unexpected char")
};

class PointMap(string[] inputLines) {
    private byte[][] inputLines = inputLines.Select(l => l.Select(ParseByte).ToArray()).ToArray();

    public int Rows = inputLines.Length;
    public int Cols = inputLines[0].Length;

    public bool IsValid(Point p) {
        return 0 <= p.row && p.row < Rows && 0 <= p.col && p.col < Cols;
    }

    public byte Char(Point p) => inputLines[p.row][p.col];
}

record Point(int row, int col) {
    public Point Up() => new(row - 1, col);
    public Point Down() => new(row + 1, col);
    public Point Left() => new(row, col - 1);
    public Point Right() => new(row, col + 1);
}

enum Direction {
    Up,
    Down,
    Left,
    Right
}

In [5]:
record PointState(Point point, Direction direction, int freeMoves);

In [6]:
(IEnumerable<Point> points, int totalCost) ShortestPath(PointMap map) {

    Point start = new(0, 0);
    PointState startState = new(start, Direction.Down, 3); // TODO do we need the other direction too?
    Point end = new(map.Rows - 1, map.Cols - 1);
    
    Dictionary<PointState, int> distances = new(map.Cols * map.Rows);
    Dictionary<PointState, PointState> previous = new(map.Cols * map.Rows);
    HashSet<PointState> queue = new();

    queue.Add(startState);
    distances[startState] = 0;

    while (queue.Count > 0) {
        var next = queue.OrderBy(p => distances[p]).First();
        var nextDist = distances[next];

        queue.Remove(next);

        var (p, dir, free) = next;
        PointState[] neighbours = (dir, free) switch {
            (Direction.Up, > 0) => [new(p.Left(), Direction.Left, 2), new(p.Right(), Direction.Right, 2), new(p.Up(), Direction.Up, free - 1) ],
            (Direction.Up, 0) => [new(p.Left(), Direction.Left, 2), new(p.Right(), Direction.Right, 2)],

            (Direction.Down, > 0) => [new(p.Left(), Direction.Left, 2), new(p.Right(), Direction.Right, 2), new(p.Down(), Direction.Down, free - 1) ],
            (Direction.Down, 0) => [new(p.Left(), Direction.Left, 2), new(p.Right(), Direction.Right, 2)],
            
            (Direction.Left, > 0) => [new(p.Up(), Direction.Up, 2), new(p.Down(), Direction.Down, 2), new(p.Left(), Direction.Left, free - 1)],
            (Direction.Left, 0) => [new(p.Up(), Direction.Up, 2), new(p.Down(), Direction.Down, 2)],

            (Direction.Right, > 0) => [new(p.Up(), Direction.Up, 2), new(p.Down(), Direction.Down, 2), new(p.Right(), Direction.Right, free - 1)],
            (Direction.Right, 0) => [new(p.Up(), Direction.Up, 2), new(p.Down(), Direction.Down, 2)],

            _ => throw new Exception("Unexpected pointState")
        };
        
        foreach (var n in neighbours) {
            if (!map.IsValid(n.point)) {
                continue;
            }

            var nDist = nextDist + map.Char(n.point);
            if (!distances.TryGetValue(n, out var nCurrent)) {
                // currently infinite distance
                distances[n] = nDist;
                previous[n] = next;
                queue.Add(n);
            } else if (nDist < nCurrent) {
                // Found shorter distance
                distances[n] = nDist;
                previous[n] = next;
            }
        }
    }

    var endStates = distances.Where(kv => kv.Key.point == end).OrderBy(kv => kv.Value);
    var best = endStates.First();

    IEnumerable<Point> PointPath(PointState point) {
        PointState p = point;

        while (p != null) {
            yield return p.point;

            previous.TryGetValue(p, out p);
        }
    }

    return (PointPath(best.Key).Reverse(), best.Value);
}

In [7]:
var testInput = """
2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
""";
var testInputLines = testInput.Split('\n');

var testInputMap = new PointMap(testInputLines);

var testResult = ShortestPath(testInputMap);

string[] Render(string[] inputLines, IEnumerable<Point> points) {
    var chars = inputLines.Select(l => l.ToCharArray()).ToArray();

        var pointList = points.ToList();

        foreach (var (p1, p2) in pointList.Zip(pointList.Skip(1))) {
            char ch = '*';
            if (p1.Down() == p2) {
                ch = 'v';
            } else if (p1.Up() == p2) {
                ch = '^';
            } else if (p1.Left() == p2) {
                ch = '<';
            } else if (p1.Right() == p2) {
                ch = '>';
            }

            chars[p2.row][p2.col] = ch;
        }

    return chars.Select(arr => new string(arr)).ToArray();
}

foreach (var line in Render(testInputLines, testResult.points)) {
    Console.WriteLine(line);
}

// This path never moves more than three consecutive blocks in the same direction and incurs a heat loss of only 102.
Console.WriteLine($"Total cost: {testResult.totalCost}");

2>>34^>>>1323
32v>>>35v>623
325524565v>54
3446585845v52
4546657867v>6
14385987984v4
44578769877v6
36378779796v>
465496798688v
456467998645v
12246868655<v
25465488877v5
43226746555v>
Total cost: 102


In [8]:
var inputMap = new PointMap(inputLines);

// This takes around 30 seconds, but it gets there in the end...
var shortestPath = ShortestPath(inputMap);
var part1Answer = shortestPath.totalCost;

Console.WriteLine(part1Answer);

866


In [9]:
// 866 is correct!
Ensure(866, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2023/day/17

In [11]:
record PointState2(Point point, Direction direction, int currentMoves, int freeMoves);

In [12]:
(IEnumerable<Point> points, int totalCost) ShortestPath2(PointMap map) {

    Point start = new(0, 0);
    PointState2 startState = new(start, Direction.Down, 0, 10); // TODO do we need the other direction too?
    Point end = new(map.Rows - 1, map.Cols - 1);
    
    Dictionary<PointState2, int> distances = new(map.Cols * map.Rows);
    Dictionary<PointState2, PointState2> previous = new(map.Cols * map.Rows);
    PriorityQueue<PointState2, int> queue = new(); // Side note: this makes a massive difference over the above

    queue.Enqueue(startState, 0);
    distances[startState] = 0;

    var currentBest = int.MaxValue; // best score to reach the end so far

    while (queue.Count > 0) {
        var next = queue.Dequeue();
        var nextDist = distances[next];

        var (p, dir, curr, free) = next;
        PointState2[] neighbours = (dir, curr, free) switch {
            (Direction.Up, < 4, _) => [new(p.Up(), dir, curr + 1, free - 1)],
            (Direction.Down, < 4, _) => [new(p.Down(), dir, curr + 1, free - 1)],
            (Direction.Left, < 4, _) => [new(p.Left(), dir, curr + 1, free - 1)],
            (Direction.Right, < 4, _) => [new(p.Right(), dir, curr + 1, free - 1)],

            (Direction.Up, _, > 0) => [new(p.Left(), Direction.Left, 1, 9), new(p.Right(), Direction.Right, 1, 9), new(p.Up(), Direction.Up, curr + 1, free - 1) ],
            (Direction.Up, _, 0) => [new(p.Left(), Direction.Left, 1, 9), new(p.Right(), Direction.Right, 1, 9)],

            (Direction.Down, _, > 0) => [new(p.Left(), Direction.Left, 1, 9), new(p.Right(), Direction.Right, 1, 9), new(p.Down(), Direction.Down, curr + 1, free - 1) ],
            (Direction.Down, _, 0) => [new(p.Left(), Direction.Left, 1, 9), new(p.Right(), Direction.Right, 1, 9)],
            
            (Direction.Left, _, > 0) => [new(p.Up(), Direction.Up, 1, 9), new(p.Down(), Direction.Down, 1, 9), new(p.Left(), Direction.Left, curr + 1, free - 1)],
            (Direction.Left, _, 0) => [new(p.Up(), Direction.Up, 1, 9), new(p.Down(), Direction.Down, 1, 9)],

            (Direction.Right, _, > 0) => [new(p.Up(), Direction.Up, 1, 9), new(p.Down(), Direction.Down, 1, 9), new(p.Right(), Direction.Right, curr + 1, free - 1)],
            (Direction.Right, _, 0) => [new(p.Up(), Direction.Up, 1, 9), new(p.Down(), Direction.Down, 1, 9)],

            _ => throw new Exception("Unexpected pointState")
        };
        
        if (next.point == start) {
            // Special case for very start
            neighbours = [new(p.Right(), Direction.Right, 1, 9), new(p.Down(), Direction.Down, 1, 9)];
        }

        if (nextDist >= currentBest) {
            // Not possible to improve, end here
            neighbours = [];
        }

        if (next.point == end && next.currentMoves >= 4) {
            // This has reached the end valid-ly
            if (distances[next] < currentBest) {
                currentBest = distances[next];
            }
        }

        foreach (var n in neighbours) {
            if (!map.IsValid(n.point)) {
                continue;
            }

            var nDist = nextDist + map.Char(n.point);
            if (!distances.TryGetValue(n, out var nCurrent)) {
                // currently infinite distance
                distances[n] = nDist;
                previous[n] = next;
                queue.Enqueue(n, nDist);
            } else if (nDist < nCurrent) {
                // Found shorter distance
                distances[n] = nDist;
                previous[n] = next;
            }
        }
    }

    var endStates = distances.Where(kv => kv.Key.point == end && kv.Key.currentMoves >= 4).OrderBy(kv => kv.Value);
    var best = endStates.First();

    IEnumerable<Point> PointPath(PointState2 point) {
        PointState2 p = point;

        while (p != null) {
            yield return p.point;

            previous.TryGetValue(p, out p);
        }
    }

    return (PointPath(best.Key).Reverse(), best.Value);
}

In [13]:
var testResult2 = ShortestPath2(testInputMap);
foreach (var line in Render(testInputLines, testResult2.points)) {
    Console.WriteLine(line);
}

// In the above example, an ultra crucible would incur the minimum possible heat loss of 94.
Console.WriteLine($"Total cost: {testResult2.totalCost}");

2>>>>>>>>1323
32154535v5623
32552456v4254
34465858v5452
45466578v>>>>
143859879845v
445787698776v
363787797965v
465496798688v
456467998645v
122468686556v
254654888773v
432267465553v
Total cost: 94


In [14]:
string[] testInputLines2 = [
"111111111111",
"999999999991",
"999999999991",
"999999999991",
"999999999991",
];

var testInputMap2 = new PointMap(testInputLines2);
var testResult22 = ShortestPath2(testInputMap2);

foreach (var line in Render(testInputLines2, testResult22.points)) {
    Console.WriteLine(line);
}

// This route causes the ultra crucible to incur the minimum possible heat loss of 71.
Console.WriteLine($"Total cost: {testResult22.totalCost}");

1>>>>>>>1111
9999999v9991
9999999v9991
9999999v9991
9999999v>>>>
Total cost: 71


In [15]:
// Directing the ultra crucible from the lava pool to the machine parts factory, what is the least heat loss it can incur?

var shortestPath2 = ShortestPath2(inputMap);
var answerPart2 = shortestPath2.totalCost;

Console.WriteLine(answerPart2);

1010


In [16]:
// 1010 is correct!
Ensure(1010, answerPart2);

In [17]:
foreach (var line in Render(inputLines, shortestPath2.points)) {
    Console.WriteLine(line);
}

1>>>>5551112233253211343462645362264333536364313674124314476621242354636547511555665246535766355535443144555253111262426511545413534355213143
2334v4145115554544563536623551634234245245226647466342141335242732317413264173727553773417346271273452636214526231261626636515443114231334134
1425v2344522223413612323412633411265511751721337441417257213276727474627134235456623341427634774763161362433662654321313511364221333134444545
2153v55141114^>>>>>>>>>>445125445152444115313243425213457743777551554161363342731541663254115427451521737561133533312121462215531232432435315
4515v13413241^534454246v126654325246235214355273527352517361157226542614511154755512673466615546552747525176152644433635456345422344511151123
2152v43114321^244465336v515334123237613537445736667574453555324626246274545121327374666676161332514434747162764221426162366535531234123343531
3442v52123155^314321635v651545632454522671676741355321433133117626761337161853414372767277524472641632243715222512245552455156241132412343151
2114v>